In [2]:
# tools for handling files
import sys
import os

# pandas/numpy for handling data
import pandas as pd
import numpy as np
from pandas import ExcelWriter
from pandas import ExcelFile

# for reading individual telomere length data from files
from ast import literal_eval

# for grabbing individual cells
import more_itertools

# my module containing functions for handling/visualizing/analyzing telomere length/chr rearrangement data
import telomere_methods_rad_patient as telo_mrp

# incase reloading modules is required
import importlib
%load_ext autoreload
%autoreload 

/usr/local/lib/python3.7/site-packages/statsmodels/compat/pandas.py:49: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  data_klasses = (pandas.Series, pandas.DataFrame, pandas.Panel)


---
&nbsp; 

...

&nbsp;

---

# Handling Telomere Length Data from TeloFISH
---

## Extracting telomere length data from all radiation therapy patients

In [18]:
all_patients_dict = telo_mrp.generate_dictionary_from_TeloLength_and_Chr_aberr_Data('../raw patient teloFISH data/')

SW9A non irrad.xlsx data extraction in progress..
BJ1 for SW9_.xlsx data extraction in progress..
SW11A non irrad.xlsx data extraction in progress..
BJ1 for SW15_.xlsx data extraction in progress..
SW6A non irrad.xlsx data extraction in progress..
SW6A irrad @ 4 Gy.xlsx data extraction in progress..
SW8B.xlsx data extraction in progress..
SW14A irrad @ 4 Gy.xlsx data extraction in progress..
SW8A irrad @ 4 Gy.xlsx data extraction in progress..
SW5A irrad @ 4 Gy.xlsx data extraction in progress..
SW8C.xlsx data extraction in progress..
SW1A non irrad.xlsx data extraction in progress..
BJ1 for SW11_.xlsx data extraction in progress..
SW16A non irrad.xlsx data extraction in progress..
BJ1 for SW13_.xlsx data extraction in progress..
BJ-hTERT for SW9_.xlsx data extraction in progress..
BJ1 for SW14_.xlsx data extraction in progress..
SW9B.xlsx data extraction in progress..
BJ1 for SW8_.xlsx data extraction in progress..
SW_1_ok_3_C_.xlsx data extraction in progress..


/usr/local/lib/python3.7/site-packages/scipy/stats/stats.py:2275: RuntimeWarning: Mean of empty slice.
  mns = a.mean(axis=axis)
/usr/local/lib/python3.7/site-packages/numpy/core/_methods.py:78: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
/usr/local/lib/python3.7/site-packages/numpy/core/_methods.py:140: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/usr/local/lib/python3.7/site-packages/numpy/core/_methods.py:110: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/usr/local/lib/python3.7/site-packages/numpy/core/_methods.py:130: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


SW3A irrad @ 4 Gy.xlsx data extraction in progress..
SW11A irrad @ 4 Gy.xlsx data extraction in progress..
BJ1 for SW16_.xlsx data extraction in progress..
BJ1 for SW12_.xlsx data extraction in progress..
SW8A non irrad.xlsx data extraction in progress..
BJ-hTERT for SW8_.xlsx data extraction in progress..
SW10A non irrad.xlsx data extraction in progress..
SW12A irrad @ 4 Gy.xlsx data extraction in progress..
SW9C.xlsx data extraction in progress..
BJ1 for SW10_.xlsx data extraction in progress..
SW7A non irrad.xlsx data extraction in progress..
SW1A irrad @ 4 Gy.xlsx data extraction in progress..
SW13A irrad @ 4 Gy.xlsx data extraction in progress..
SW1B.xlsx data extraction in progress..
BJ-hTERT for SW6_.xlsx data extraction in progress..
SW13B.xlsx data extraction in progress..
BJ1 for SW2_.xlsx data extraction in progress..
SW2A non irrad.xlsx data extraction in progress..
SW5C.xlsx data extraction in progress..
SW15C.xlsx data extraction in progress..
SW7C.xlsx data extraction in

## Making dataframe from dict w/ all patients telomere length data, contains telo means & individual telos as list

In [7]:
# making df w/ all telomere data
all_patients_df = telo_mrp.generate_dataframe_from_dict_and_generate_histograms_stats(all_patients_dict)

# counting telomeres per quartile
all_patients_df = telo_mrp.calculate_apply_teloQuartiles_dataframe(all_patients_df)
all_patients_df['Q1'] = all_patients_df['Q1'].astype('float64')
all_patients_df['Q2-3'] = all_patients_df['Q2-3'].astype('float64')
all_patients_df['Q4'] = all_patients_df['Q4'].astype('float64')

# don't need telo means per cell @ this time
all_patients_df = all_patients_df.drop(['cell data'], axis=1)

print(all_patients_df.shape)
all_patients_df.head(4)

To display graphs pass the value "yes graphs" to the function otherwise default option="no graphs"
(59, 9)


,patient id,timepoint,telo data,chr data,status,telo means,Q1,Q2-3,Q4
0,1,1 non irrad,0 78.134078 1 82.357542 2 ...,chr data,IT WORKS PEGGY <333,84.796738,1195.0,2214.0,1191.0
1,1,2 irrad @ 4 Gy,0 137.262570 1 115.089385 2 ...,chr data,IT WORKS PEGGY <333,90.975987,724.0,2343.0,1533.0
2,1,3 B,0 95.027933 1 93.972067 2 ...,chr data,IT WORKS PEGGY <333,116.780229,231.0,1448.0,2921.0
3,1,4 C,0 124.592179 1 134.062806 2 ...,chr data,IT WORKS PEGGY <333,99.346663,372.0,2210.0,2018.0


## Saving all patients telomere length data for later retrieval

In [8]:
# changing telo data to list in prep for saving to csv

all_patients_df['telo data'] = all_patients_df['telo data'].apply(lambda row: row.tolist())
all_patients_df.to_csv('../compiled patient data csv files/all_patients_df.csv', index=False)

## Generating all patients telo df containing telo counts per quartile melted into tidy data format 

In [9]:
melted_all_patients_df = pd.melt(
    all_patients_df,
    id_vars = [col for col in all_patients_df.columns if col != 'Q1' and col != 'Q2-3' and col != 'Q4'],
    var_name='relative Q',
    value_name='Q freq counts')

melted_all_patients_df['Q freq counts'] = melted_all_patients_df['Q freq counts'].astype('float64')
melted_all_patients_df.head(4)

,patient id,timepoint,telo data,chr data,status,telo means,relative Q,Q freq counts
0,1,1 non irrad,"[78.13407813845714, 82.35754182161699, 30.6201...",chr data,IT WORKS PEGGY <333,84.796738,Q1,1195.0
1,1,2 irrad @ 4 Gy,"[137.26256970269498, 115.0893853661058, 89.748...",chr data,IT WORKS PEGGY <333,90.975987,Q1,724.0
2,1,3 B,"[95.02793287109652, 93.97206695030657, 185.832...",chr data,IT WORKS PEGGY <333,116.780229,Q1,231.0
3,1,4 C,"[124.59217865321546, 134.06280604091415, 45.40...",chr data,IT WORKS PEGGY <333,99.346663,Q1,372.0


## Saving melted all patients df to csv

In [10]:
melted_all_patients_df.to_csv('../compiled patient data csv files/melted_all_patients_df.csv', index=False)

## Pivoted Dataframe w/ timepoints as columns, and telomere length means for each patient timepoint in rows

In [11]:
pivot_patients_telo_means_df = all_patients_df.pivot(index='patient id', columns='timepoint', values='telo means')
pivot_patients_telo_means_df = pivot_patients_telo_means_df.drop(13)
pivot_patients_telo_means_df.head(4)

timepoint,1 non irrad,2 irrad @ 4 Gy,3 B,4 C
patient id,,,,
1,84.796738,90.975987,116.780229,99.346663
2,119.774143,133.199629,159.828115,108.915685
3,83.351204,87.295754,101.433049,95.669819
5,85.506373,113.096219,118.340987,97.832439


## Saving pivoted telo means df to file

In [12]:
pivot_patients_telo_means_df.to_csv('../compiled patient data csv files/pivot_patients_telo_means_df.csv', index=False)

## Exploding individual telomere length measurements from contained list into dataframe (i.e row per individual telomere) while retaining related column info

In [15]:
# can imagine the lists containing the individual telos per patient exploding to the right; maintains the index relationship
explode_telos_raw = all_patients_df['telo data'].apply(pd.Series)

print(explode_telos_raw.shape)
explode_telos_raw.head(4)

(59, 4600)


,0,1,2,3,4,5,6,7,8,9,...,4590,4591,4592,4593,4594,4595,4596,4597,4598,4599
0,78.134078,82.357542,30.620112,158.379888,68.631285,154.156424,70.743017,85.525140,66.519553,53.849162,...,97.139665,138.318436,70.743017,77.078212,39.067039,87.636871,71.798883,63.351955,120.368715,48.569832
1,137.262570,115.089385,89.748603,90.804469,80.245810,99.251397,80.245810,68.631285,46.458101,104.530726,...,60.184357,116.145251,74.966480,68.631285,96.083799,74.966480,83.413408,73.910614,81.301676,81.301676
2,95.027933,93.972067,185.832402,117.201117,121.424581,211.173184,91.860335,98.195531,117.201117,85.525140,...,116.145251,131.983240,80.245810,122.480447,85.525140,147.821229,133.039106,92.916201,54.905028,114.033519
3,124.592179,134.062806,45.402235,97.139665,70.743017,59.128492,103.474860,74.966480,131.983240,59.128492,...,101.363128,83.413408,134.094972,140.430167,155.212290,62.296089,107.698324,103.474860,179.011213,158.755090


In [16]:
exploded_telos_all_patients_df = (explode_telos_raw
                                  
    # we'll merge the exploded telos df w/ our original all patients df on the index!
    .merge(all_patients_df, right_index = True, left_index = True)
    .drop(['telo data', 'Q1', 'Q2-3', 'Q4'], axis = 1)
    .melt(id_vars = ['patient id', 'timepoint', 'chr data', 'status', 'telo means'], value_name = "telo data exploded") 
    .drop("variable", axis = 1)
    .dropna())

# exploded_telos_all_patients_df['telo data exploded'] = exploded_telos_all_patients_df['telo data exploded'].astype('float64')
exploded_telos_all_patients_df.head(4)

,patient id,timepoint,chr data,status,telo means,individual telomeres
0,1,1 non irrad,chr data,IT WORKS PEGGY <333,84.796738,78.134078
1,1,2 irrad @ 4 Gy,chr data,IT WORKS PEGGY <333,90.975987,137.262570
2,1,3 B,chr data,IT WORKS PEGGY <333,116.780229,95.027933
3,1,4 C,chr data,IT WORKS PEGGY <333,99.346663,124.592179


## Saving exploded telomere df for later retrieval

In [17]:
exploded_telos_all_patients_df.to_csv('../compiled patient data csv files/exploded_telos_all_patients_df.csv', index=False)

# Handling Telomere Length Data from qPCR - Aidan & Lynn
---

## Extracting average telomere length data by qPCR data from Aidan & Lynn

In [28]:
all_qPCR_df = pd.read_excel('../qPCR telo data/REVISED Tel +Alb (both plates) 2019-08-05 13 Quantification Cq Results.xlsx',
                            sheet_name=1,
                           skiprows=1,
                            usecols=[24, 25, 26,]
                           )

all_qPCR_df.columns = ['Sample', 'telo means qPCR', 'SEM']
all_qPCR_df['Sample'] = all_qPCR_df['Sample'].astype('str')

In [29]:
all_qPCR_df.dropna(axis=0, inplace=True)
all_qPCR_df.drop([24, 47, 48], axis=0, inplace=True)

In [30]:
all_qPCR_df['Sample'] = all_qPCR_df['Sample'].apply(lambda x: telo_mrp.change_sample_ID(x))
all_qPCR_df['timepoint'] = all_qPCR_df['Sample'].apply(lambda x: telo_mrp.make_timepoint_col(x))
all_qPCR_df['patient id'] = all_qPCR_df['Sample'].apply(lambda x: telo_mrp.make_patient_ID(x))

all_qPCR_df['patient id'] = all_qPCR_df['patient id'].astype('str')
all_qPCR_df.drop(['Sample'], axis=1, inplace=True)

In [33]:
cols = ['patient id', 'timepoint', 'telo means qPCR', 'SEM']
all_qPCR_df = all_qPCR_df.reindex(columns=cols)
all_qPCR_df.head(4)

,patient id,timepoint,telo means qPCR,SEM
0,1,1 non irrad,2.182028,0.083233
1,1,3 B,1.964348,0.051942
2,1,4 C,1.749799,0.068195
3,2,1 non irrad,1.669445,0.034776


In [34]:
all_qPCR_df.to_csv('../qPCR telo data/all_qPCR_df.csv', index=False)

# Handling Chromosome Aberration Data from Subtelo-dGH 
---

In [32]:
# %load_ext autoreload

%autoreload

In [33]:
all_chr_aberr_df = telo_mrp.make_dataframe_chr_aberr_data('../dGH scoresheets/')

<DirEntry 'SW3_timepoints_subtelodGH.xlsx'>
<DirEntry 'SW5_timepoints_subtelodGH.xlsx'>
<DirEntry 'SW2_timepoints_subtelodGH.xlsx'>
<DirEntry 'SW1_timepoints_subtelodGH.xlsx'>
<DirEntry 'SW6_timepoints_subtelodGH.xlsx'>


In [34]:
all_chr_aberr_df.head(4)

,patient id,timepoint,cell number,chromosome,metaphase size,inversion size,# inversions,inversion notes,terminal inversion size,# terminal inversions,...,# 5+ chr sat. associations,dicentrics,tricentrics,chr fragments,expected chr fragments,excess chr fragments,translocation intra notes,"translocations reciprocal 1,2,3","translocations one-way 1,2,3",sample notes
0,6,1 non irrad,1,1,medium,NaN,0,NaN,NaN,0,...,0,0,0,0,0,0,NaN,0,0,NaN
1,6,1 non irrad,1,2,medium,NaN,0,NaN,NaN,0,...,0,0,0,0,0,0,NaN,0,0,NaN
2,6,1 non irrad,1,3,medium,NaN,0,NaN,NaN,0,...,0,0,0,0,0,0,NaN,0,0,NaN
3,6,1 non irrad,2,1,medium,NaN,0,NaN,NaN,0,...,0,0,0,0,0,0,NaN,0,0,NaN


In [35]:
all_chr_aberr_df.to_csv('../compiled patient data csv files/all_chr_aberr_df.csv', index=False)